# **Notebook 1 - Road Networks Extraction using OSMnx**

---

This notebook handles the download, preparation, and storage of road networks from **OpenStreetMap (OSM)** using the **OSMnx** Python package. It represents the first step of the DiverCity workflow.
Starting from the geographic center of each city, the script retrieves a drivable road network within a fixed radius, applies standard simplifications, and saves the resulting graph for further processing.

---

### **Parameters Used in This Study:**  
- **Location:** Defined by geographic coordinates of the city center, sourced from [latlong.net](https://www.latlong.net/).  
- **Radius:** **30 km (30,000 meters)**, ensuring comprehensive coverage of the urban area. TIP: consider adding a 1-2km buffer to minimize border effects.
- **Network Type:** `drive`, focusing on car navigable roads.  
- **Simplification:** `True`, simplifying the graph by removing intermediate nodes while preserving connectivity. This reduces computational complexity for subsequent analysis.  

---


In [ ]:
import os
import osmnx as ox
import gzip
import pandas as pd

from my_utils import create_folder_if_not_exists
from network_measures import compute_network_measures

## **1. Parameters**

The parameters below define the configuration for downloading and storing a city’s road network.
Modify them to extract networks from different locations or to test alternative configurations. <br>
TIP: Consider storing city coordinates in `../data/city_info.csv` instead of hard-coding values here.


In [ ]:
# Identifier for the network (used for naming output files)
city_id = "milan"

# Option 1 - Recommended: Load coordinates from a CSV file (../data/city_info.csv)
# The CSV should contain at least: "city", "lat", "lng"
df_cities = pd.read_csv("../data/city_info.csv")
city_center = df_cities[df_cities["city"] == city_id][["lat", "lng"]].values[0]

# Option 2 - Manual specification (uncomment to use)
# city_center = (45.4642, 9.19)  # Example: Milan

radius_m = 10_000   # Extraction radius in meters around the city center
network_type = "drive"  # Type of network ('drive', 'walk', 'bike', 'all')
simplify = True  # Simplify the network to remove non-intersection nodes

# Destination folder and paths for saving the network data
dest_folder = "../data/road_networks"  # Folder to store network files
dest_path_gz = f"{dest_folder}/{city_id}_{network_type}_{radius_m}.graphml.gz"
dest_path = f"{dest_folder}/{city_id}_{network_type}_{radius_m}.graphml"
save_as_gzip = True  # Save compressed (.gz) file to reduce storage size

# Create the destination folder if it doesn't already exist
create_folder_if_not_exists(dest_folder)

## **2. Road Network download & visualization**

In [ ]:
# Retrieve a network centered on the selected city within the specified radius
G = ox.graph_from_point(city_center, dist=radius_m, network_type=network_type, simplify=simplify)

# Impute missing edge speeds and calculate edge travel times
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

# Plot a preview of the network for verification
ox.plot_graph(G, node_size=0, show=True, close=True);

## **3. Save the Road Network**

In [ ]:
# Save as standard GraphML file (uncompressed)
ox.save_graphml(G, dest_path)

# Optionally compress the saved file to reduce storage space
if save_as_gzip:
    with open(dest_path, 'rb') as f_in:
        with gzip.open(dest_path_gz, 'wb') as f_out:
            f_out.writelines(f_in)
    
    # Remove the uncompressed file after compressing
    os.remove(dest_path)

## **4. Compute road network measures**

This section analyzes the structure and properties of the road network. Specifically, it will:  
- Compute **basic network statistics** such as node count, edge count, and density.  
- Analyze **edge characteristics** to understand connectivity patterns.  
- Extract **attractor-related information**, identifying key edges and their influence.  

The computed network measures will be stored in a structured dictionary format, with the following key components:  

```python
dict_measures = {
    "metadata": ...,          # General information about the computation (e.g., version, date)
    "basic_stats": ...,       # Summary statistics of the road network (e.g., number of nodes, edges, density)
    "info_attractors": {"edges": ...},      # Information related to attractor edges in the network
    "info_edges": ...         # Detailed attributes of individual edges in the network
}


In [ ]:
# Create the output folder
folder_info = f"../data/road_networks_info/{city_id}/"
os.makedirs(folder_info, exist_ok=True)

dict_network_measures = compute_network_measures(city_id, city_center, 
                                                 "drive", radius_m, 
                                                 save=True, saveFig=False, filename=folder_info)